Criar uma tabela de Clientes para equipe de marketing:<br>
Importante somente pessoas que tenham e-mail e tefefone cadastrados

In [0]:
%sql
USE CATALOG bikestore;

In [0]:
# Definir pastas do projetos em variaveis para facilitar
bronze_path   = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/bronze/'
silver_path   = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/silver/'
gold_path     = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/gold/'
resource_path = 'abfss://data-ext-azure@externalazure25.dfs.core.windows.net/bikestore/resource/origem'
resource_path_volume = '/Volumes/bikestore/logistics/bikestore_rescource/origem/'

In [0]:
# criar varias tabelas temporárias de forma prática
bronze_map = {
    ##"tmp_bronze_brands":      f"{bronze_path}/brands/",
    ##"tmp_bronze_categories":  f"{bronze_path}/categories/",
    "tmp_bronze_customers":   f"{bronze_path}/customers/",
    #"tmp_bronze_order_items": f"{bronze_path}/order_details/",
    #"tmp_bronze_orders":      f"{bronze_path}/orders/",
    #"tmp_bronze_products":    f"{bronze_path}/products/",
    #"tmp_bronze_staffs":      f"{bronze_path}/staffs/",
    #"tmp_bronze_stocks":      f"{bronze_path}/stocks/",
    #"tmp_bronze_stores":      f"{bronze_path}/stores/",
}
for view_name, path in bronze_map.items():
    (spark.read.format('delta')
        .load(path)
        .createOrReplaceTempView(view_name))
 


In [0]:
%python
df_customer_silver = spark.sql("""
    SELECT 
    C.customer_id,
    C.first_name,
    C.last_name,
    C.phone,
    C.email,
    C.street,
    C.city,
    C.state,
    C.zip_code
    FROM tmp_bronze_customers AS C
    WHERE  1 = 1
    AND C.phone IS NOT NULL ---vazio
    AND C.phone NOT IN ('NULL', 'NULL ') --texto null
    AND C.email IS NOT NULL
    AND C.email NOT IN ('NULL', 'NULL ')
                              
                              """)

# salvar em Delta na silver 
df_customer_silver.write\
    .mode('overwrite')\
    .format('delta')\
    .option('mergeSchema','true')\
    .save(f'{silver_path}/customer')
